In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
!pip install nlpaug sacremoses transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 


In [17]:
import nlpaug.augmenter.word as naw

back_translation_aug_de = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en')

In [12]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/sat.csv")

In [13]:
row = df.shape[0]
col = df.shape[1]

In [21]:
de_bt = []
for r in range(row):
    for c in range(col):
      bt = back_translation_aug_de.augment(df.iloc[r].iat[c])
      de_bt.append(bt)

In [23]:
import csv
with open("drive/MyDrive/backtranslation_de.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(de_bt)

In [24]:
back_translation_aug_ru = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-ru',
    to_model_name='facebook/wmt19-ru-en')

In [25]:
ru_bt = []
for r in range(row):
    for c in range(col):
      bt = back_translation_aug_ru.augment(df.iloc[r].iat[c])
      ru_bt.append(bt)

In [26]:
with open("drive/MyDrive/backtranslation_ru.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(ru_bt)

In [35]:
original = []
for r in range(row):
    for c in range(col):
      original.append(df.iloc[r].iat[c])

I am sorry that I cannot detect any defense mechanism from our conversation. If you don't mind, are you willing to play it again or end the conversation now?
That's ok, thanks. I know this may be hard for you to discuss but your information has been very useful. Has you had a real fight with {}?
This is really helpful, thank you. Has your childhood-self treated {} in an excessively friendly manner?
Cool, thank you for answering that. I know it isn't always easy, but anything you can tell me will help me to better understand your situation. Do you think that the feelings/attitudes that your childhood-self has about {} are similar to any of their primary caregivers (such as parents or relatives who take the most of the responsibility of taking care of them during childhood)?
Thank you for your response, I would really like to help. Has your childhood-self behaviors become more childlike or primitive with respect to {} ?
Good job! Thank you so much for your cooperation. Then I was wonderi

In [42]:
with open("drive/MyDrive/user_rewritings.csv", 'w') as f:
    writer = csv.writer(f)
    for val in original:
        writer.writerow([val])

In [44]:
!pip install pylev sentence_transformers language_tool_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=12a5c1164e2824b740bf4e7a441a6564656edef4be728ab5e17be9ce5a05fce0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [45]:
import pylev
from sentence_transformers import SentenceTransformer, util
import language_tool_python
import torch

# initializing LanguageTool and SentenceTransformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
sen_model = SentenceTransformer('paraphrase-MiniLM-L3-v2').to(device)
tool = language_tool_python.LanguageTool('en-US')

def get_distance(src_txt, paraphrased_txt):
    """Returns levenschtein distance at word level between src_text and paraphrase"""
    return pylev.levenschtein(src_txt.split(), paraphrased_txt.split())

def get_similarity(src_txt, paraphrased_txt):
    """Returns cosine similarity between source and paraphrase sentence vectors"""
    src_txt_encoded = sen_model.encode(src_txt, convert_to_tensor=True)
    paraphrased_txt_encoded = sen_model.encode(paraphrased_txt, convert_to_tensor=True)
    return util.pytorch_cos_sim(src_txt_encoded , paraphrased_txt_encoded).item()

def get_num_grammatical_errors(paraphrased_txt):
    """Returns the number of errors calculated by LanguageTool"""
    return len(tool.check(paraphrased_txt))

Downloading:   0%|          | 0.00/690 [00:01<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp466urbgx.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [53]:
user_rewrite = pd.read_csv("drive/MyDrive/user_rewritings.csv", header=None,)

bt_de = pd.read_csv("drive/MyDrive/backtranslation_de.csv", header=None,)
bt_ru = pd.read_csv("drive/MyDrive/backtranslation_ru.csv", header=None,)

In [54]:
distance_de = []
similarity_de = []
error_de = []

for i in range(len(user_rewrite[0])):
    distance_de.append(get_distance(user_rewrite[0][i], bt_de[0][i]))
    similarity_de.append(get_similarity(user_rewrite[0][i], bt_de[0][i]))
    error_de.append(get_num_grammatical_errors(bt_de[0][i]))


In [55]:
distance_ru = []
similarity_ru = []
error_ru = []

for i in range(len(user_rewrite[0])):
    distance_ru.append(get_distance(user_rewrite[0][i], bt_ru[0][i]))
    similarity_ru.append(get_similarity(user_rewrite[0][i], bt_ru[0][i]))
    error_ru.append(get_num_grammatical_errors(bt_ru[0][i]))


In [61]:
bt_de.columns = ['utterance', 'distance', 'similarity', 'grammatical_errors']

In [59]:
bt_de['distance'] = distance_de
bt_de['similarity'] = similarity_de
bt_de['grammatical_errors'] = error_de


In [63]:
bt_ru['distance'] = distance_ru
bt_ru['similarity'] = similarity_ru
bt_ru['grammatical_errors'] = error_ru
bt_ru.columns = ['utterance', 'distance', 'similarity', 'grammatical_errors']

In [67]:
bt_ru.to_csv("drive/MyDrive/bt_ru.csv", index=False)

In [68]:
bt_de.to_csv("drive/MyDrive/bt_de.csv", index=False)